In [2]:
import numpy as np
import sys,os
import matplotlib.pyplot as plt
from scipy.stats import multivariate_t, norm, chi2, multivariate_normal
import sklearn
from sklearn.covariance import GraphicalLasso
import scipy
from sklearn.metrics import zero_one_loss
from sklearn.metrics.pairwise import pairwise_kernels
import time
import tqdm
import networkx as nx
import scipy.integrate as integrate
from sklearn.datasets import make_sparse_spd_matrix
sys.path.insert(0, 'C:/Users/User/Code/DyGraph')

import DyGraph as dg

In [58]:
G =nx.erdos_renyi_graph(20, 0.25, seed=42)
A = np.array(nx.adjacency_matrix(G).todense())
A[np.tril_indices(20)] = 0
A = np.multiply(A, np.random.uniform(-0.6,0.6, size = (20,20)))
A = A+A.T
np.fill_diagonal(A, np.sum(np.abs(A),axis=1))


C:\Users\User\AppData\Local\Temp\ipykernel_20876\1600366158.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = np.array(nx.adjacency_matrix(G).todense())


In [59]:
np.linalg.eigh(A)

(array([0.08431162, 0.33742154, 0.44081946, 0.51545803, 0.58086229,
        0.70447216, 0.85743763, 0.93608348, 1.11565385, 1.20309048,
        1.53734358, 1.60560262, 1.89368973, 2.03790044, 2.20598245,
        2.46349333, 2.94656048, 3.42355666, 3.71285753, 4.12644555]),
 array([[ 2.77301034e-02,  1.70220849e-01, -1.58640737e-01,
         -7.67901165e-02, -5.04197723e-02,  1.49984032e-01,
         -8.43269661e-02,  5.94944532e-03, -1.50629099e-01,
         -4.28116914e-01,  2.91804371e-01,  4.22880685e-02,
          7.55200824e-02,  4.90852526e-01, -3.88270035e-01,
          8.73394795e-02, -2.10488562e-01,  4.05252185e-01,
          2.30407586e-02, -3.20592231e-03],
        [-5.71072998e-03, -1.44929824e-01, -1.84697086e-01,
         -5.97094214e-02,  2.24914075e-02, -9.45961836e-02,
         -6.76628451e-01, -5.27710884e-02, -3.55690482e-01,
          1.22174753e-01, -1.40482876e-01,  9.45517558e-02,
         -1.11615119e-01, -3.24854279e-01, -3.80716721e-01,
          4.92615177e-

In [64]:
print(np.round(A[:,1],2))
A2 = A.copy()
A2_SIGNS = np.sign(A2)
A2 = np.power(np.abs(A2), 1.2)
A2 = A2*A2_SIGNS
np.fill_diagonal(A2,np.diag(A))
np.round(A2[:,1],2)

X1 = multivariate_normal.rvs(mean = np.zeros(20), cov = np.linalg.inv(A), size = 30 ) 
X2 = multivariate_normal.rvs(mean = np.zeros(20), cov = np.linalg.inv(A2), size = 130 ) 
X = np.vstack((X1,X2))
X.shape

[ 0.    1.31  0.42 -0.   -0.   -0.   -0.26  0.   -0.    0.23  0.4   0.
  0.   -0.    0.   -0.   -0.    0.    0.   -0.  ]


(160, 20)

In [2]:
def generate_skew_t(Sigma, gamma, nu, n):

    

    d = Sigma.shape[0]
    x = np.random.multivariate_normal(mean = np.zeros(d),cov = Sigma, size = n)
    y = np.zeros(shape = (n, d))

    for i in range(n):
        
        u = np.random.uniform()
        v = chi2.ppf(u, df = nu)/nu 

        u = np.random.uniform()
        y[i] = (1/v)*gamma+ np.sqrt(1/v)*x[i]

    return y

In [3]:
prec_1 = np.array([[ 1.51515152,  0.        , -1.06060606,  0.55151515],
                    [ 0.        ,  2.5       ,  0.        ,  0.        ],
                    [-1.06060606,  0.        ,  2.24242424, -0.60606061],
                    [ 0.55151515,  0.        , -0.60606061,  1.51515152]])

    
gamma = np.zeros(4)# 0.000001*np.ones(4)
nu = 4
n = 1000
X = generate_skew_t(Sigma = np.linalg.inv(prec_1), gamma =  gamma, nu = nu,n = n)
#X = multivariate_t.rvs(shape = np.linalg.inv(prec_1), df = 4, size = 100)
obs_per_graph = 10

In [6]:
# X = generate_skew_t(Sigma = np.linalg.inv(prec_1), gamma =  gamma, nu = nu,n = n)
d = X.shape[1]
iteration = 0
U = np.random.normal(size = (4, 4))
theta_pre = prec_1 #np.identity(4)
gamma_pre = np.zeros(4)
nr_itr = 10
while iteration < nr_itr:
    # E-step

    M = np.einsum('nj,jk,nk->n', X, theta_pre, X)  # Mahalanobis distance
    tau = (nu + d)/(nu + M)
    tau_inv = (nu + M)/(nu + d-1)
    # update gamma
    gamma_pre =  np.sum(X, axis=0)/np.sum(tau_inv) 
    # update theta
    gamma_tau = sum([tau_inv[i]*np.outer(gamma_pre, gamma_pre) for i in range(n)])
    s_1= sum([np.outer(X[i], gamma_pre) for i in range(n)])/float(X.shape[0])
    s_2 = sum([np.outer(gamma_pre,X[i]) for i in range(n)])/float(X.shape[0])
    S = np.einsum('nj,n,nk->jk', X, tau, X)/float(X.shape[0]) - s_1 -s_2 + gamma_tau 
    theta_pre = np.linalg.inv(S)

    iteration += 1



In [5]:
theta_pre

array([[ 0.76100783, -0.06479157, -0.46324138,  0.86375057],
       [-0.06479157,  2.58478275,  0.0559423 , -0.01303167],
       [-0.46324138,  0.0559423 ,  1.90145551, -0.83742821],
       [ 0.86375057, -0.01303167, -0.83742821,  1.37581723]])

In [19]:
gamma_pre

array([-2.44785590e-02, -1.83281111e-02, -4.39429385e-02, -9.26051770e-05])

In [18]:
np.round(theta_pre,2)

array([[ 1.6 ,  0.1 , -0.98,  0.54],
       [ 0.1 ,  2.12, -0.8 ,  0.11],
       [-0.98, -0.8 ,  1.13, -0.4 ],
       [ 0.54,  0.11, -0.4 ,  1.41]])

In [336]:
gamma_pre

array([-0.0324536 ,  0.01492607,  0.00282577, -0.02887333])

In [256]:
x = np.random.gamma(scale=1/2, shape=3, size= 100)
print(np.mean(x))
print(3/2)
print("\n")
print(np.mean(1/x))
print(2/2)

1.5942486296996785
1.5


1.044787653154836
1.0


In [218]:
gamma_pre

array([-0.02035587,  0.04222638,  0.09791859,  0.01934823])

In [193]:
#theta_pre = np.identity(4)
#gamma_pre = np.zeros(4)
nr_itr = 1000
iteration = 0
while iteration < nr_itr:
    # E-step
    x = X
    M = np.einsum('nj,jk,nk->n', x, theta_pre, x)  # Mahalanobis distance
    tau = (nu + d)/(nu  + M)
    tau_inv = (nu  + M)/(nu + d-1)
    gamma_tau = sum([tau_inv[i]*np.outer(gamma_pre, gamma_pre) for i in range(n)])
    gamma_pre =  sum([np.dot(theta_pre, X[i]) for i in range(n)])/np.sum(tau_inv)
    S = np.einsum('nj,n,nk->jk', x, tau, x)/float(x.shape[0]) + sum([np.outer(X[0], gamma_pre) for i in range(n)])/float(x.shape[0]) + gamma_tau 
    theta_pre = np.linalg.inv(S)
    # gamma_pre =  sum([np.dot(theta_pre, X[i]) for i in range(n)])/np.sum(tau_inv)


    iteration += 1

In [194]:
theta_pre

array([[ 0.57789328,  0.30525655,  0.02131161,  0.09422571],
       [ 0.28104568,  1.24705279, -0.49542212,  0.01215562],
       [ 0.0355943 , -0.48446141,  0.35448143, -0.03299859],
       [ 0.09238286,  0.03853138, -0.04939271,  0.9054783 ]])

In [61]:
theta_pre

array([[ 400.7111662 ,  -30.44524071, -380.2974025 ,  244.32279263],
       [ -30.39771615,    6.97679141,  -20.91887137,  -13.9533675 ],
       [-380.28512841,  -20.8104357 ,  891.00552616, -280.64602941],
       [ 244.30896237,  -13.9849621 , -280.62263314,  236.88638325]])

In [52]:
theta_pre

array([[ 443.0133114 , -104.93596066, -440.72230393,  289.46535903],
       [-104.93299927,  171.46904426,  -67.18435874,  -91.41312691],
       [-440.7178959 ,  -67.17236299,  668.12766756, -317.38521714],
       [ 289.46461549,  -91.41530613, -317.38544917,  301.76563468]])